<img src="img/logo_demcompare.png" width="100" align="right">

# Demcompare: reprojection and coregistration tutorial


This notebook is an introduction to demcompare and its coregistration step.

#### Imports and external functions

In [ ]:
import pyproj # pyproj as first import is necessary 

In [ ]:
from snippets.utils_notebook import *

In [ ]:
import bokeh
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import row, column
import xarray as xr
import numpy as np

## What is demcompare ? 

* Demcompare is a python software that aims at comparing two DEMs together.
* It performs the coregistration based on the Nuth & Kääb universal coregistration method.
* Two steps are available in demcompare coregistration's step: reprojection and coregistration 
* It provides a wide variety of standard metrics which can be classified (not shown in this Notebook).

## Context

During the optional coregistration step, demcompare performs the coregistration on two uncoregistered DEMs like the ones below

Superposition of two DEMs that need to be coregistered
<img src="img/doc_ref.gif" width="300" align="center">

## Glossary

**DEM (Digital Elevation Model)** : a 3D computer graphics representation of elevation data to represent terrain.

**Coregistration** :  this step looks for offsets differences and align DEMs together

## Coregistration in demcompare

This diagram shows the coregistration steps and variables that take place during demcompare's coregistration : 

* Firstly, both input dems input_ref and input_sec are reprojected to a common resolution and size, obtaining the dems input_reproj_ref and input_reproj_sec
* Once the two reprojected dems are obtained, the coregistration algorithm computes the coregistration offsets (x_off, y_off) , and creates the Transformation object that is given as an output.
* During the coregistration algorithm, the reprojected and coregistered dems input_reproj_coreg_ref and input_reproj_coreg_sec are also computed. It is to be noticed that whilst both dems share the same origin, this may not be the input_ref origin. For this reason, those dems are only meant to be used for the altitude difference and statistics computation.
* If the original input_sec is to be coregistered, then the coreg_sec = Transformation.apply(input_sec) function is to be used. It will indeed apply the coregistration offsets to the input dem without altering its resolution or size.

<img src="img/schema_coreg.png" width="800">

## Inputs

* For the coregistration step, two DEMs are necessary
    * input_ref and input_sec
    * input_sec is to be coregistered on input_ref
    * The inputs can have different size and resolution. 
    * By default, demcompare considers that the reference DEM has better resolution. 

The user sets demcompare with a json configuration file. Each DEM is introduced with a dictionary. All dictionnary's possibilities are described in the next chunk.

|           Name           |                  Description                  |  Type  | Default value | Required |
|:------------------------:|:---------------------------------------------:|:------:|:-------------:|:--------:|
|     _path_               |             Path of the input Ref             | string |      None     |    Yes   |
|     _zunit_              |          Z axes unit of the input Ref         | string |       m       |    No    |
| _geoid_georef_           | True if the georef of the input Ref is geoid  |  bool  |     False     |    No    |
|  _geoid_path_            |          Geoid path of the input Ref          | string |      None     |    No    |
|    _nodata_              |         No data value of the input Ref        |   int  |      None     |    No    |
|  _classification_layers_ |        Path to the classification layer       | string |      None     |    No    |

A possible configuration for inputs is presented here. 

In [ ]:
input_ref = {
            "path" : "data/grenoble/Copernicus_DSM_10_N45_00_E005_00_DEM.tif",
            "zunit" : "m",
    }

input_sec = {
            "path" : "data/grenoble/Copernicus_blurred_and_shifted.tif",
            "zunit" : "m",
            "nodata" : -32768
    }

DEMs are loaded with the load_dem function from dem_tools. The loaded DEMs are stored as input_sec and input_ref.

In [ ]:
from demcompare.dem_tools import load_dem

# load dems 
input_sec  = load_dem(path=input_sec["path"], zunit=input_sec["zunit"])
input_ref  = load_dem(path=input_ref["path"], zunit=input_ref["zunit"])

DEMs are stored in an xarray.Dataset with demcompare's dataset structure.  Demcompare's dataset is described in the next chunk.

| **Dataset's element** |                    **Definition**                    |
|:---------------------:|:----------------------------------------------------:|
| **Dimension/Coordinate**                                                     |
|          row          |                     number of row                    |
|          col          |                   number of columns                  |
|       trans_len       |         indice link to georef_transform datas        |
|    **georef_transform**                                                      |
|           c           |         x-coordinate of the upper left pixel         |
|           a           |   pixel size in the x-direction in map units/pixel   |
|           b           |                 rotation about x-axis                |
|           f           |         y-coordinate of the upper left pixel         |
|           d           |                 rotation about y-axis                |
|           e           | pixel size in the y-direction in map units, negative |
|       **attributes**                                                         |
|         nodata        |                  image nodata value                  |
|       input_img       |                   image input path                   |
|          crs          |                       image crs                      |
|          xres         |         x resolution (value of transform[1])         |
|          yres         |         y resolution (value of transform[5])         |
|       plani_unit      |             georefence's planimetric unit            |
|         zunit         |               input image z unit value               |
|         bounds        |                     image bounds                     |
|    source_rasterio    |        rasterio's DatasetReader object or None       |
|       geoid_path      |                      geoid path                      |

In [ ]:
input_ref

Here, one can visualize the superposition of both DEMs and notice the offsets between them. They also differ in size and resolution. 

In [ ]:
show(stack_dems(input_ref, input_sec, "Originals DEMS"))

# Demcompare's coregistration pipeline

# Reprojection step

This step reprojects both DEMs into the same resolution and size. The common resolution is defined by the parameter sampling_source. By default, the parameter sampling_source is set to "sec". That means that both reprojected DEMs will have the sec's resolution, and hence the ref will be interpolated during reprojection. The size of the reprojected DEMs is their common georeferenced intersection.     

**Warning** : This step is automatically handled by the coregistration class. The next 3 chunks are only here to show the reprojection step importance.  

The function reproject_dems from dem_tools is called for the purposes of the notebook.

In [ ]:
from demcompare.dem_tools import reproject_dems

We reproject the inputs in the space 

In [ ]:
reproj_sec, reproj_ref, _ = reproject_dems(input_sec, input_ref)

Here, you can see both DEMS in the same resolution and size

In [ ]:
show(stack_dems(reproj_sec, reproj_ref, "Reprojected DEMS"))

# Coregistration step

We call the coregistration class

In [ ]:
from demcompare.coregistration import Coregistration

The user sets demcompare with a json configuration file. Each pipeline's step is introduced with a dictionary. All possibilities are described in the next chunk.

|             Name            |                          Description                          |  Type  |    Default Value   | Required |
|:---------------------------:|:-------------------------------------------------------------:|:------:|:------------------:|----------|
|        _method_name_        |               Planimetric coregistration method               | string | nuth_kaab_internal | No       |
|    _number_of_iterations_   |       Number of iterations of the coregistration method       |   int  |          6         | No       |
| _estimated_initial_shift_x_ |            Estimated initial x coregistration shift           |   int  |          0         | No       |
| _estimated_initial_shift_y_ |            Estimated initial y coregistration shift           |   int  |          0         | No       |
|      _sampling_source_      |                Sampling source for reprojection               | string |         sec        | No       |
| _save_optional_outputs_ | If save coregistration method outputs such as iteration plots | string |        False       | No       |

A possible configuration for inputs is presented here. 

In [ ]:
cfg = {
    "coregistration": {
        "method_name": "nuth_kaab_internal", #one method available for now
        "number_of_iterations": 6,
        "estimated_initial_shift_x": 0,
        "estimated_initial_shift_y": 0,
    }
}

We create coregistration object

In [ ]:
coregistration_ = Coregistration(cfg["coregistration"])

## Apply transformation to original sec DEM

The coregistration is computed and results are stored in transformation.

In [ ]:
transformation = coregistration_.compute_coregistration(input_sec, input_ref)

Different transformation's attributes are printed

In [ ]:
print(transformation)

The offsets are applied to original second dem 

In [ ]:
coreg_sec = transformation.apply_transform(input_sec)

Here, you can visualize Reference DEM with the coregistered second DEM.

In [ ]:
show(stack_dems(input_ref, coreg_sec, "Referenced DEM and coregistered Second DEM"))

# Compute altitude differences

* Demcompare also computes the altitude differences between DEM

We access Demcompare's reprojected DEMs in order to compute altitude difference.

In [ ]:
reproj_ref = coregistration_.reproj_ref
reproj_sec = coregistration_.reproj_sec

We access Demcompare's reprojected and coregistered DEMs in order to compute altitude difference.

In [ ]:
reproj_coreg_ref = coregistration_.reproj_coreg_ref
reproj_coreg_sec = coregistration_.reproj_coreg_sec

We use the function compute_dems_diff from dem_tools

In [ ]:
from demcompare.dem_tools import compute_dems_diff

First with the reprojected DEMs we obtain the initial altitude difference

In [ ]:
altitude_diff_before_coreg = compute_dems_diff(reproj_sec, reproj_ref)

And with the coregistered reprojected DEM we obtain the final altitude difference

In [ ]:
altitude_diff_after_coreg = compute_dems_diff(reproj_coreg_ref, reproj_coreg_sec)

You can visualize the altitude differences between the DEMs before and after the coregistration

In [ ]:
altitude_diff_after_coreg

In [ ]:
show(side_by_side_fig(altitude_diff_before_coreg, 
                altitude_diff_after_coreg,
                "Initial altitude difference",
                "Final altitude difference"))

# Some precision about reprojection

User can inverse ref and sec for the reprojection with  "sampling_source" parameter. But he has to be careful. **Indeed, using reference dem with low resolution can alter coregistration results.** 

Reminder of coregistraion steps:

<img src="img/schema_coreg.png" width="800">

With default `sampling_source` parameter, the reference dem will be reprojected to the resolution and size of the second dem.

But another possible configuration for inputs is presented here, with `sampling_source` parameter set to "ref". 

In [ ]:
cfg = {
    "coregistration": {
        "method_name": "nuth_kaab_internal", #one method available for now
        "number_of_iterations": 6,
        "estimated_initial_shift_x": 0,
        "estimated_initial_shift_y": 0,
        # sampling source default value is "sec"
        "sampling_source": "ref"
    }
}

New coregistration object

In [ ]:
coregistration_samp_ref = Coregistration(cfg["coregistration"])

Compute coregistration

In [ ]:
transformation_samp_ref = coregistration_samp_ref.compute_coregistration(input_sec, input_ref)

Different transformation's attributes are printed. Here, the difference between the offsets obtained with the sampling source parameter as ref and sec is not big but notice that it can be.

In [ ]:
print(transformation_samp_ref)

The offsets are applied 

In [ ]:
coreg_sec_samp_ref = transformation_samp_ref.apply_transform(input_sec)

We can visualize the effects of reprojection with some internals outputs

In [ ]:
reproj_coreg_ref_samp_source_ref = coregistration_samp_ref.reproj_coreg_ref
reproj_coreg_sec_samp_source_ref = coregistration_samp_ref.reproj_coreg_sec

reproj_coreg_ref_samp_source_sec = coregistration_.reproj_coreg_ref
reproj_coreg_sec_samp_source_sec = coregistration_.reproj_coreg_sec

reproj_ref_samp_source_ref = coregistration_samp_ref.reproj_ref
reproj_sec_samp_source_ref = coregistration_samp_ref.reproj_sec

Internal reprojection with default configuration

In [ ]:
show(side_by_side_fig(reproj_coreg_ref_samp_source_ref, 
                 reproj_coreg_sec_samp_source_ref, 
                 "Internal reprojection reference dem with sampling source as ref",
                 "Internal reprojection second dem with sampling source as ref"))

Internal reprojection with sampling_source parameter to "ref"

In [ ]:
show(side_by_side_fig(reproj_coreg_ref_samp_source_sec, 
                 reproj_coreg_sec_samp_source_sec, 
                 "Internal reprojection reference dem with sampling source as ref",
                 "Internal reprojection second dem with sampling source as ref"))

We calculate the altitude difference with the reference dem as sampling source

In [ ]:
altitude_diff_samp_source_before = compute_dems_diff(reproj_ref_samp_source_ref, reproj_sec_samp_source_ref)
altitude_diff_samp_source_after = compute_dems_diff(reproj_coreg_ref_samp_source_ref, reproj_coreg_sec_samp_source_ref)

In [ ]:
show(side_by_side_fig(altitude_diff_samp_source_before, 
                 altitude_diff_samp_source_after, 
                 "Initial altitude difference",
                 "Final altitude difference"))

To visualize outputs with the new configuration we use the altitude difference of the coreg_reproj DEMs. We can notice the difference in the resolution between both altitude differences.

In [ ]:
show(side_by_side_fig(altitude_diff_samp_source_after, 
                 altitude_diff_after_coreg, 
                 "Altitude difference sampling_source ref",
                 "Altitude difference sampling_source sec"))